# deploy model to azure ml to predict Active Power for Wind Turbines at Windy Hill Wind Turbine farm

This example is less about the model (arguably you could do this a number of ways) and more about a hello world example using azure ml

Portions (mostly) from Azure documentation, todo includes service security

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.85


In [106]:
# package for parsing objects to/from model
!pip install inference_schema

  Created wheel for wrapt: filename=wrapt-1.11.1-cp36-cp36m-linux_x86_64.whl size=66554 sha256=a898d5e1501c6f8d99e39ebfb47955f39988ff2b858916fe529d520d6650d27e
  Stored in directory: /home/azureuser/.cache/pip/wheels/89/67/41/63cbf0f6ac0a6156588b9587be4db5565f8c6d8ccef98202fc
Successfully built wrapt
ERROR: astroid 2.3.1 has requirement six==1.12, but you'll have six 1.14.0 which is incompatible.
  Found existing installation: wrapt 1.11.2
    Uninstalling wrapt-1.11.2:
      Successfully uninstalled wrapt-1.11.2


# configure environment, load model and test

In [13]:
# configuration
model_name = "Wind_Air_4.pkl"
workspace = 'turbineActivePower'

In [2]:
# load created model that was previously created and registered

from azureml.core import Workspace
from azureml.core.model import Model
import os 

ws = Workspace.from_config()
model=Model(ws, workspace)
model.download(target_dir=os.getcwd(), exist_ok=True)

# verify the downloaded model file
file_path = os.path.join(os.getcwd(), model_name)
os.stat(file_path)

os.stat_result(st_mode=33279, st_ino=5290, st_dev=46, st_nlink=1, st_uid=0, st_gid=0, st_size=828, st_atime=1585550695, st_mtime=1585559031, st_ctime=1585559031)

In [629]:
# validate the model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import pickle
import joblib

# sample prediction
new_input = [[45, 6.6]] #Temp=45 F, Wind Speed = 6.6 m/s
poly_features = PolynomialFeatures(degree=4)

loaded_model = joblib.load( os.path.join(os.getcwd(),model_name))
output = loaded_model.predict(poly_features.fit_transform(new_input))[0]
output

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


297.87831929324454

# Create scoring code - to run model

In [356]:
%%writefile score.py
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import json
import pickle
import numpy as np
import pandas as pd
import os
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model
import joblib

from inference_schema.parameter_types.standard_py_parameter_type import StandardPythonParameterType
from inference_schema.schema_decorators import input_schema, output_schema

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'Wind_Air_4.pkl')
    model = joblib.load(model_path)

    
#describe input
features = {
    'Air Temperature': 45,
    'Wind Speed': 6.6,
}
# describe output
prediction = {
    'Version': 1,
    'Prediction': 297.878
}

@input_schema('param', StandardPythonParameterType(features))
#@output_schema(StandardPythonParameterType(prediction))
    
def run(param):
    try:
        air_temp = param['Air Temperature']
        wind_speed = param['Wind Speed']
        # run model
        poly_features = PolynomialFeatures(degree=4)
        result = model.predict(poly_features.fit_transform([[air_temp,wind_speed]]))
        # clip Active Power values - not ideal, todo: improve/resolve this(tm)
        # cut-in speed for power generation
        if wind_speed < 2.5: 
            prediction = 0
        # compensating for the model, see todo above:
        elif wind_speed > 13.25:
            prediction = 1432
        else:
            prediction = result[0]
        return {'Prediction': prediction, 'Version': 2,"result": result[0]}
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


# Create scoring code - to run model - this time modified to use schema supported by Power BI

In [673]:
%%writefile score.py
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import json
import pickle
import numpy as np
import pandas as pd
import os
import azureml.train.automl
from sklearn.externals import joblib
from azureml.core.model import Model
import joblib
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.schema_decorators import input_schema, output_schema

def init():
    global model
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'Wind_Air_4.pkl')
    model = joblib.load(model_path)

# describe input
features = pd.DataFrame(data=[{
    'Air Temperature': 45.5,
    'Wind Speed': 6.6
}])

# describe output
active_power_prediction = np.array([297.878])

@input_schema('data', PandasParameterType(features))
@output_schema(NumpyParameterType(active_power_prediction))
  
def run(data):
    output = np.empty([0])
    try:
        for index,row in data.iterrows():
            # clip Active Power values - not ideal, todo: improve/resolve this(tm)
            # cut-in speed for power generation
            if row['Wind Speed'] < 2.5: 
                prediction = 0.0
            # compensating for the model, see todo above:
            elif row['Wind Speed'] > 13.25:
                prediction = 1432.0
            else:
                poly_features = PolynomialFeatures(degree=4)
                prediction = model.predict(poly_features.fit_transform([[row['Air Temperature'],row['Wind Speed']]]))[0]
            output = np.append(output,prediction)
        return output.tolist()
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


In [671]:
# load the script
%run -i score.py

In [672]:
# test script
# will need to comment out @output line from score.py
input_data = [{'Air Temperature': 45.1,'Wind Speed': 6.6},{'Air Temperature': 48,'Wind Speed': 9}]
#input_data = [{'Air Temperature': 45.1,'Wind Speed': 6.6}]
#Note: for production format is: input_data = {'param': {'Air Temperature': 45.1,'Wind Speed': 20}}
data = json.dumps(input_data)
prediction = None
result = run(input_data)
print(result)

[298.14948763009147, 738.0379427055908]


In [528]:
# configure environment to run model and view

from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("scikit-learn==0.22.1")
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("azureml-defaults>=1.0.45")
myenv.add_pip_package("inference-schema[numpy-support]")
myenv.add_pip_package("azureml-train")
myenv.add_pip_package("azureml.train.automl")
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("azureml-telemetry")
myenv.add_pip_package("joblib")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [189]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
  - scikit-learn==0.22.1
  - inference-schema[numpy-support]
  - azureml-train
  - azureml.train.automl
  - azureml-defaults
  - azureml-telemetry
  - joblib
channels:
- conda-forge



In [163]:
# create configuration file
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Wind Turbine",  "method" : "sklearn"}, 
                                               description='Predict Wind Turbine Active Power')

## Create service

In [165]:
# create service
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='turbine-svc', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Running........................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 610 ms, sys: 73.4 ms, total: 684 ms
Wall time: 10min 31s


In [674]:
# update scoring code (only required when code changes are made)
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
service.update(inference_config=inference_config)
service.wait_for_deployment(show_output=True)

Running......
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [ ]:
# display logs
logs = service.get_logs()
print(logs)

## swagger definition

In [604]:
import pprint as pprint
pprint.pprint(json.loads(requests.get(service.swagger_uri).text))

{'consumes': ['application/json'],
 'definitions': {'ErrorResponse': {'properties': {'message': {'type': 'string'},
                                                  'status_code': {'format': 'int32',
                                                                  'type': 'integer'}},
                                   'type': 'object'},
                 'ServiceInput': {'example': {'data': [{'Air Temperature': 45.5,
                                                        'Wind Speed': 6.6}]},
                                  'properties': {'data': {'items': {'properties': {'Air Temperature': {'format': 'double',
                                                                                                       'type': 'number'},
                                                                                   'Wind Speed': {'format': 'double',
                                                                                                  'type': 'number'}},
                  

## get service URL and query

In [ ]:
print(service.scoring_uri)

In [680]:
import requests
import json

input_data = {'data': [{'Air Temperature': 45.1,'Wind Speed': 6.6},{'Air Temperature': 45.1,'Wind Speed': 6.6}]}
data = json.dumps(input_data)

# for AKS deployment you'd need to the service key in the header as well
#api_key = service.get_key()
#headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 
headers = {'Content-Type':'application/json'} 

resp = requests.post(service.scoring_uri, data, headers=headers)
print(resp)
print("prediction:", resp.text)

<Response [200]>
prediction: [298.14948763009147, 298.14948763009147]


In [675]:

headers = {'Content-Type':'application/json'} 
for wind_speed in range (0,16):
    input_data = {'data': [{'Air Temperature': 45.1,'Wind Speed': wind_speed}]}
    data = json.dumps(input_data)
    resp = requests.post(service.scoring_uri, data, headers=headers)
    print(f'{resp},{resp.text}')

<Response [200]>,[0.0]
<Response [200]>,[0.0]
<Response [200]>,[0.0]
<Response [200]>,[22.953421462315305]
<Response [200]>,[76.1170121699904]
<Response [200]>,[137.48869821448181]
<Response [200]>,[226.5663501563904]
<Response [200]>,[354.10469893181187]
<Response [200]>,[522.1153358523433]
<Response [200]>,[723.8667126050703]
<Response [200]>,[943.8841412525784]
<Response [200]>,[1157.9497942329467]
<Response [200]>,[1333.1027043597564]
<Response [200]>,[1427.6387648220666]
<Response [200]>,[1432.0]
<Response [200]>,[1432.0]


In [133]:
# clean-up
#service.delete()